# Prep textgrids and convert to dataframes

In [4]:
import pandas as pd
import os
from audiolabel import read_label
import re
import numpy as np
from syltippy import syllabize

Import `syll_dict.txt` which contains GlobalPhone transcriptions for each syllable produced in the data set.

In [9]:
syllable_dictionary = pd.read_csv("data/syllable_dictionary.txt", sep = "\t", header = None)
syllable_dictionary = syllable_dictionary.rename(columns = {0: "each_syllable", 1: "GP_syll"})
syllable_dictionary.head()

,each_syllable,GP_syll
0,vos,b o s
1,nos,n o s
2,bar,b a rf
3,gua,G w a
4,ja,x a


In [14]:
# create function that will take in textgrid and output data frame including stress assignment
def tg2df_stress(filepath, wd_tier, ph_tier, syll):
    
    [wddf, phdf] = read_label(filepath,
        ftype='praat',
        tiers=[wd_tier, ph_tier]
    )

    # Throw an error if tiers are not strictly hierarchical.
    # words contain phones
    assert(wddf.t1.isin(phdf.t1).all())
    assert(wddf.t2.isin(phdf.t2).all())
    
    # Add phone duration and speaker
    phdf['dur_ph'] = phdf.t2 - phdf.t1

    # add filename to dataframe
    phdf['Participant'] = filepath[9:13]
    
    if filepath[9]=="s":
        phdf['Corpus'] = "DIMEx100"
    else:
        phdf['Corpus'] = "CBAS"
    
    # Merge phone and word tiers.
    newdf = pd.merge_asof(
        phdf.rename({'t1': 't1_ph', 't2': 't2_ph', "label": "phone"}, axis='columns'),
        wddf.drop(["fname"], axis = 1) \
        .rename({'t1': 't1_wd', 't2': 't2_wd', "label": "word"}, axis='columns'),
        left_on='t1_ph',
        right_on='t1_wd'
    )

    # Add word-init and -final columns
    newdf['is_wdinit_ph'] = newdf.t1_ph == newdf.t1_wd
    newdf['is_wdfin_ph'] = newdf.t2_ph == newdf.t2_wd
        
    # drop rows where word=="" or ".sil"
    newdf = newdf[(newdf["word"]!="") &
                (newdf["word"]!=".sil") &
             (newdf["word"]!=".bn")]

    # convert v_7 to accent over vowel
    # fix notation in dimex corpus, where V_7 yields accented V
    newdf['word'] = newdf['word'].apply(lambda x: re.sub("a_7", "á", x))
    newdf['word'] = newdf['word'].apply(lambda x: re.sub("i_7", "í", x))
    newdf['word'] = newdf['word'].apply(lambda x: re.sub("o_7", "ó", x))
    newdf['word'] = newdf['word'].apply(lambda x: re.sub("u_7", "ú", x))
    newdf['word'] = newdf['word'].apply(lambda x: re.sub("e_7", "é", x))

    # fix tildas
    newdf['word'] = newdf['word'].apply(lambda x: re.sub("n~", "ñ", x))
        
    # obtain syllables for each word
    newdf["syllable"] = newdf["word"].apply(lambda x: syllabize(x)[0])
    newdf["stress_ind"] = newdf['word'].apply(lambda x: syllabize(x)[1])
    newdf = newdf.reset_index(drop = True)
    len_nd = len(newdf)
    
    # one observation for each word
    words_sylls = newdf[['word', 't1_wd', 'stress_ind', 'syllable']].copy()
    words_sylls = words_sylls.drop_duplicates(subset = ["t1_wd","word"])

    # create new df with each syllable contained in each word
    syll_cols = pd.DataFrame({'word':np.repeat(words_sylls.word.values, words_sylls.syllable.str.len()),
                          'stress_ind': np.repeat(words_sylls.stress_ind.values, words_sylls.syllable.str.len()),
                          't1_wd': np.repeat(words_sylls.t1_wd.values, words_sylls.syllable.str.len()),
                              'each_syllable':np.concatenate(words_sylls.syllable.values)})
    syll_cols['each_syllable'] = syll_cols["each_syllable"].apply(lambda x: x[1:] if x.startswith("h") else x)
    syll_cols = syll_cols.reset_index(drop = True)

    # create new df that contains GP for each syllable
    stress_finder = syll_cols.merge(syll, how = "left", on = "each_syllable")

    # now create syllable index for each syllable
    stress_finder["syll_ind"] = stress_finder.groupby(["word", "t1_wd"]).cumcount()

    # now create column to indicate whether syllable is stressed
    stress_finder['is_stress'] = np.where(stress_finder.stress_ind == stress_finder.syll_ind, 1, 0)

    # create list from GP_syll column
    stress_finder['syll_list'] = stress_finder['GP_syll'].apply(lambda x: str(x).split(" "))

    # now create new columns
    phones_stressed = pd.DataFrame({'word':np.repeat(stress_finder.word.values, stress_finder.syll_list.str.len()),
                                'is_stress': np.repeat(stress_finder.is_stress.values, stress_finder.syll_list.str.len()),
                                    't1_wd': np.repeat(stress_finder.t1_wd.values, stress_finder.syll_list.str.len()),
                        'GP_ph':np.concatenate(stress_finder.syll_list.values)})  
    phones_stressed = phones_stressed.reset_index(drop = True)
    
    # count number of phones in each word
    counts = phones_stressed.groupby(['word', 't1_wd'])['GP_ph'].size().reset_index(name = 'num_ph')
    counts = counts.sort_values(by = ["t1_wd"], ignore_index = True)
    phones_stressed = phones_stressed.merge(counts, on = ["word", 't1_wd'])
    len_ph = len(phones_stressed)
    
    # add 'num_ph' to new df with info from newdf
    testdf = phones_stressed.drop(['is_stress', 'GP_ph'], axis = 1)
    testdf = testdf.drop_duplicates()
    test_3 = newdf.merge(testdf, on = ["word", 't1_wd'])
    
    # count number of times each word appears
    word_counts = test_3.groupby(['word', 't1_wd']).size().reset_index(name = "count_word")
    word_counts = word_counts.sort_values(by = ["t1_wd"], ignore_index = True)
    test_3 = test_3.merge(word_counts, on = ["word", 't1_wd'])
    
    # define list of words to be removed and remove them
    removed = test_3[test_3['num_ph']!=test_3['count_word']]
    test_3 = test_3[test_3['num_ph']==test_3['count_word']]
    test_3 = test_3.reset_index(drop = True)
    rem_words = list(removed['word'].unique())
    
    # drop removed words from phones_stressed then merge
    phones_stressed = phones_stressed[~phones_stressed['word'].isin(rem_words)]
    phones_stressed = phones_stressed.reset_index(drop = True)
    test_3 = test_3.drop(['phone'], axis = 1)
    final_df = test_3.join(phones_stressed, how = "inner", rsuffix = "_dict")
    final_df = final_df.drop(['stress_ind', "syllable", "count_word", "num_ph", "word_dict", "t1_wd_dict", "num_ph_dict"], axis = 1)
    len_ph_rem = len(phones_stressed)
    
    #final_df = final_df.drop_duplicates()
    len_final = len(final_df)
    
    if len_ph != len_nd:
        print("length error with ", filepath)
        if len_final == len_ph_rem:
            print("resolved and following words dropped: ",rem_words)
            return(final_df)
        else:
            print('issue remains')
            return(final_df)
    
    else:
        return(final_df)

In [15]:
test = tg2df_stress("data/tgs/p111_spanish3.TextGrid", 'default - words', 'default - phones', syllable_dictionary)
test.to_csv("data/test.csv", index = False)

Now create a for loop to iterate through text grids and produce df with all data from all speakers.

In [16]:
tg_folder = "data/tgs/"
corpora = os.listdir(tg_folder)
df_list = []

for f in corpora:
    if f.endswith(".TextGrid"):
        filepath = tg_folder + f
    
        if f.startswith("p"):
            wd_tier = "default - words"
            ph_tier = "default - phones"
        else:
            wd_tier = "word"
            ph_tier = "phone"
    
        new_df = tg2df_stress(filepath, wd_tier, ph_tier, syllable_dictionary)
        df_list.append(new_df)
        display(new_df.tail(1))

tg_data = pd.concat(df_list)

,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
1416,469.795,469.985,data/tgs/p111_spanish1.TextGrid,0.19,p111,CBAS,469.195,469.985,tribalismo,False,True,0,o


,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
762,315.582,315.812,data/tgs/p111_spanish2.TextGrid,0.23,p111,CBAS,314.902,315.812,conveniente,False,True,0,e


,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
298,103.794,103.924,data/tgs/p111_spanish3.TextGrid,0.13,p111,CBAS,103.414,103.924,zurdo,False,True,0,o


,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
1455,500.71,500.89,data/tgs/p113_spanish1.TextGrid,0.18,p113,CBAS,499.97,500.89,tribalismo,False,True,0,o


,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
833,248.651,248.851,data/tgs/p113_spanish2.TextGrid,0.2,p113,CBAS,247.711,248.851,conveniente,False,True,0,e


...

length error with  data/tgs/s05648.TextGrid
resolved and following words dropped:  ['tasas']


,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
44,4.044,4.13,data/tgs/s05648.TextGrid,0.086,s056,DIMEx100,3.664,4.13,tiempo,False,True,0,o


length error with  data/tgs/s05649.TextGrid
resolved and following words dropped:  ['dar']


,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
66,4.727,4.873,data/tgs/s05649.TextGrid,0.146,s056,DIMEx100,3.947,4.873,correspondientes,False,True,0,s


,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
82,5.584,5.622,data/tgs/s05650.TextGrid,0.038,s056,DIMEx100,5.185,5.622,sector,False,True,1,rf


In [21]:
tg_data.tail()

,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph
75,4.930,5.003,data/tgs/s05650.TextGrid,0.073,s056,DIMEx100,4.519,5.076,novedades,False,False,0,e
76,5.003,5.076,data/tgs/s05650.TextGrid,0.073,s056,DIMEx100,4.519,5.076,novedades,False,True,0,k
77,5.185,5.289,data/tgs/s05650.TextGrid,0.104,s056,DIMEx100,5.185,5.622,sector,True,False,1,t
78,5.289,5.368,data/tgs/s05650.TextGrid,0.079,s056,DIMEx100,5.185,5.622,sector,False,False,1,o
79,5.368,5.432,data/tgs/s05650.TextGrid,0.064,s056,DIMEx100,5.185,5.622,sector,False,False,1,rf


In [17]:
tg_data.GP_ph.unique()

array(['b', 'a', 'x', 'o', 'T', 'm', 'i', 'a+', 's', 'k', 'l', 'i+', 't',
       'e', 'r', 'j', 'rf', 'G', 'u', 'n', 'w', 'o+', 'ng', 'L', 'p',
       'u+', 'tS', 'g', 'd', 'aI', 'oI', 'f', 'n~', 'e+', 'aU', 'z',
       'nan'], dtype=object)

In [18]:
# fix notation of vowels, where v+ yields v
tg_data['GP_ph'] = tg_data['GP_ph'].apply(lambda x: x[0] if (x.startswith(("a", "e", "i", "o", "u")) & x.endswith("+")) else x)

In [19]:
tg_data['GP_ph'].unique()

array(['b', 'a', 'x', 'o', 'T', 'm', 'i', 's', 'k', 'l', 't', 'e', 'r',
       'j', 'rf', 'G', 'u', 'n', 'w', 'ng', 'L', 'p', 'tS', 'g', 'd',
       'aI', 'oI', 'f', 'n~', 'aU', 'z', 'nan'], dtype=object)

In [20]:
# rename T to s
tg_data['GP_ph'] = tg_data['GP_ph'].apply(lambda x: "s" if x == "T" else x)

In [21]:
tg_data["GP_ph"].unique()

array(['b', 'a', 'x', 'o', 's', 'm', 'i', 'k', 'l', 't', 'e', 'r', 'j',
       'rf', 'G', 'u', 'n', 'w', 'ng', 'L', 'p', 'tS', 'g', 'd', 'aI',
       'oI', 'f', 'n~', 'aU', 'z', 'nan'], dtype=object)

In [22]:
vowels = ["a", "e", "i", "o", "u", 'aI', 'oI', 'aU']
tg_data["is_vowel"] = np.where(tg_data.GP_ph.isin(vowels), 1, 0)
tg_data.sample(50)

,t1_ph,t2_ph,fname,dur_ph,Participant,Corpus,t1_wd,t2_wd,word,is_wdinit_ph,is_wdfin_ph,is_stress,GP_ph,is_vowel
235,93.081,93.141,data/tgs/p114_spanish2.TextGrid,0.060,p114,CBAS,92.501,93.651,televangelista,False,False,0,e,1
591,311.056,311.166,data/tgs/p122_spanish1.TextGrid,0.110,p122,CBAS,310.736,311.166,dios,False,True,1,s,0
579,227.616,227.706,data/tgs/p121_spanish2.TextGrid,0.090,p121,CBAS,227.076,227.956,resolver,False,False,1,b,0
885,338.092,338.322,data/tgs/p114_spanish3.TextGrid,0.230,p114,CBAS,337.672,338.322,hondos,False,True,0,s,0
9,0.675,0.723,data/tgs/s00239.TextGrid,0.048,s002,DIMEx100,0.596,1.300,facturación,False,False,0,a,1
309,170.480,170.640,data/tgs/p114_spanish1.TextGrid,0.160,p114,CBAS,170.110,170.840,comerás,False,False,1,a,1
71,19.370,19.550,data/tgs/p123_spanish3.TextGrid,0.180,p123,CBAS,19.010,19.550,corta,False,True,0,a,1
515,201.646,201.716,data/tgs/p124_spanish3.TextGrid,0.070,p124,CBAS,201.646,202.386,convicción,True,False,0,k,0
387,220.783,220.873,data/tgs/p117_spanish1.TextGrid,0.090,p117,CBAS,220.463,220.973,causal,False,False,1,a,1
97,77.880,77.930,data/tgs/p117_spanish2.TextGrid,0.050,p117,CBAS,77.560,78.290,terminarás,False,False,0,n,0


In [23]:
vowels = tg_data[tg_data["is_vowel"]==1].copy()
vowels.shape

(18189, 14)

In [24]:
vowels.groupby(['Corpus', "GP_ph"]).count()

t1_ph  t2_ph  fname  dur_ph  Participant  t1_wd  t2_wd  word  \
Corpus   GP_ph                                                                 
CBAS     a       4531   4531   4531    4531         4531   4531   4531  4531   
         aI        47     47     47      47           47     47     47    47   
         aU        20     20     20      20           20     20     20    20   
         e       2774   2774   2774    2774         2774   2774   2774  2774   
         i       1562   1562   1562    1562         1562   1562   1562  1562   
         o       3072   3072   3072    3072         3072   3072   3072  3072   
         oI        24     24     24      24           24     24     24    24   
         u        586    586    586     586          586    586    586   586   
DIMEx100 a       1564   1564   1564    1564         1564   1564   1564  1564   
         aU         1      1      1       1            1      1      1     1   
         e       1831   1831   1831    1831         1831   1831   1831  1831   
         i        656    656    656     656          656    656    656   656   
         o       1191   1191   1191    1191         1191   1191   1191  1191   
         u        330    330    330     330          330    330    330   330   

                is_wdinit_ph  is_wdfin_ph  is_stress  is_vowel  
Corpus   GP_ph                                                  
CBAS     a              4531         4531       4531      4531  
         aI               47           47         47        47  
         aU               20           20         20        20  
         e              2774         2774       2774      2774  
         i              1562         1562       1562      1562  
         o              3072         3072       3072      3072  
         oI               24           24         24        24  
         u               586          586        586       586  
DIMEx100 a              1564         1564       1564      1564  
         aU                1            1          1         1  
         e              1831         1831       1831      1831  
         i               656          656        656       656  
         o              1191         1191       1191      1191  
         u               330          330        330       330

In [25]:
tg_data.to_csv("data/tg_data.csv", index = False)